# ETF Portfolio Simulation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In this notebook, we examine a simple portfolio consisting of two ETFs. We begin by exploring how to download data for multiple tickers and retrieve their associated metadata.

In [ ]:
ticker_1_str = "SWDA.MI"
ticker_2_str = "XGLE.MI"

ticker_1 = yf.Ticker(ticker_1_str)
ticker_2 = yf.Ticker(ticker_2_str)

print(f"Going to download {ticker_1.info["shortName"]} and {ticker_2.info["shortName"]}")

In [ ]:
def yf_download(ticker, start="1900-01-01"):
    #df = yf.download(ticker, start=start, multi_level_index=True, auto_adjust=True)
    df = yf.download(ticker, start=start, multi_level_index=True, auto_adjust=True)
    return df

In [ ]:
# Stock (iShares Core MSCI World) + Bond (Eurozone Government Bond) UCITS ETFs
df = yf_download([ticker_1_str, ticker_2_str], start="2010-01-01")
df.info()

We focus on close values and perform an exploratory analysis:

In [ ]:
df_cl = df["Close"]
df_cl.describe()

SWDA has more values than XGLE. The difference is composed of null values. We can propagate the last valid price.

In [ ]:
print(f"Going to forward-fill null values: {df_cl.isnull().values.sum()}")
df_cl = df_cl.ffill()
print(f"Residual null values: {df_cl.isnull().values.sum()}")

In [ ]:
# Normalized exploratory plot
(df_cl / df_cl.iloc[0]).plot()

## Historical Portfolio Scenarios - No Rebalancing


Consider a portfolio built using only the two ETFs downloaded above. The portfolio has a stock component equal to ```stock_alloc```.

We start by evaluating its historical performance with no rebalancing:

In [ ]:
def eval_portfolio(df2, stock_alloc):
    df = df2.copy()

    s_t0 = df.iloc[0] 
    n_t0 = np.array([stock_alloc, 1 - stock_alloc]) / s_t0
    
    portfolio = []
    for i in range(len(df)):
        value_i = np.sum(n_t0 * df.iloc[i])
        portfolio.append(value_i)

    df[f"{stock_alloc:.0%}"] = portfolio
    return df

To compare the performance in different allocation scenarios, we could:

1. Call the simple ```eval_portfolio``` multiple times and aggregate the result;
2. Modify the ```eval_portfolio``` method to handle a list of ```stock_alloc```. 

The second solutions is more efficient, but let's start with the first approach for simplicity.

In [ ]:
stock_allocs = np.linspace(0, 1, num=5)

evaluations = [eval_portfolio(df_cl, alloc).iloc[:,2] for alloc in stock_allocs]
evaluations = pd.concat(evaluations, axis=1)
evaluations.plot()